## Exploratory Data Analysis

Use this notebook to get familiar with the datasets we have. There is 10 questions we need to answer during the EDA.


We shouldn't limit our EDA to these 10 questions. Let's be creative :).

In [5]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2

In [87]:
#opening connection & cursor
conn = psycopg2.connect(
                            host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                            database="mid_term_project",
                            user="lhl_student",
                            password="lhl_student")
cursor = conn.cursor()
test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 10000;"

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

df = pd.DataFrame()

for i in range(1, 13):
    if i == 1:
        df = create_pandas_table(test2)
    else:
        test_pd_table = create_pandas_table(test2)
        df = pd.concat([df, test_pd_table]) 
        print(i)

# remember to close connection to allow for others to use the bandwidth
cursor.close()
conn.close()
df.head() # show table querried

2
3
4
5
6
7
8
9
10
11
12


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-02-01,AA,AA_CODESHARE,AA,4946,PT,N614AE,4946,10434,AVP,...,104.0,2.0,0.0,0.0,0.0,85.0,None,NaN,NaN,None
1,2019-02-01,AA,AA,AA,224,AA,N189AN,224,14100,PHL,...,1303.0,0.0,0.0,25.0,0.0,0.0,None,NaN,NaN,None
2,2019-02-01,AA,AA,AA,227,AA,N902AN,227,13204,MCO,...,2218.0,6.0,0.0,0.0,0.0,15.0,None,NaN,NaN,None
3,2019-02-01,AA,AA,AA,228,AA,N70020,228,13930,ORD,...,235.0,30.0,0.0,21.0,0.0,0.0,None,NaN,NaN,None
4,2019-02-01,AA,AA,AA,259,AA,N947AN,259,10721,BOS,...,867.0,4.0,0.0,0.0,0.0,19.0,None,NaN,NaN,None


In [72]:
df.to_csv('flights2.csv')

In [73]:
df.isnull().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier                 0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                    0
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
dep_delay                   0
taxi_out                    0
wheels_off                  0
wheels_on                   0
taxi_in                     0
crs_arr_time                0
arr_time                    0
arr_delay                   0
cancelled                   0
cancellation_code      120000
diverted                    0
dup                         0
crs_elapsed_time            1
actual_elapsed_time         0
air_time                    0
flights                     0
distance                    0
carrier_de

In [74]:
df.cancelled

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9995    0.0
9996    0.0
9997    0.0
9998    0.0
9999    0.0
Name: cancelled, Length: 120000, dtype: float64

Chicago, IL              7187
Atlanta, GA              5722
Denver, CO               4882
Dallas/Fort Worth, TX    4208
Houston, TX              4070
Los Angeles, CA          3802
New York, NY             3720
Washington, DC           3268
Charlotte, NC            3250
Phoenix, AZ              3164
Name: origin_city_name, dtype: int64

In [108]:
df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [88]:
df.shape

(120000, 42)

KeyError: 'origin_country_name'

#### **Task 1**: Test the hypothesis that the delay is from Normal distribution. and that **mean** of the delay is 0. Be careful about the outliers.

In [119]:
df['arr_delay'].agg(['max', 'min', 'mean', 'std', 'median'])

max       199.000000
min       -87.000000
mean        3.211583
std        32.456724
median     -6.000000
Name: arr_delay, dtype: float64

In [120]:
df['arr_delay'].sort_values()

5573    -87.0
6114    -70.0
5676    -69.0
9281    -66.0
6096    -63.0
        ...  
382     199.0
2772    199.0
2069    199.0
4955    199.0
1473    199.0
Name: arr_delay, Length: 120000, dtype: float64

In [94]:
len(df[df['arr_delay'] > 200])

0

In [48]:
stat, p = shapiro(df.arr_delay)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

stat=0.763, p=0.000
Probably not Gaussian


In [97]:
tset, pval = ttest_1samp(df.arr_delay, 0)

In [98]:
print("p-values",pval)
if pval < 0.05:    # alpha value is 0.05 or 5%
    print(" we are rejecting null hypothesis")
else:
    print("we are accepting null hypothesis")

p-values 0.0
 we are rejecting null hypothesis


#### **Task 2**: Is average/median monthly delay different during the year? If yes, which are months with the biggest delays and what could be the reason?

In [15]:
df.sort_values('fl_date')

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-01-01,AA,AA_CODESHARE,AA,5621,OH,N720PS,5621,11057,CLT,...,490.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
6663,2019-01-01,G4,G4,G4,711,G4,311NV,711,12889,LAS,...,954.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
6664,2019-01-01,G4,G4,G4,1202,G4,217NV,1202,10431,AVL,...,519.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
6665,2019-01-01,G4,G4,G4,1301,G4,273NV,1301,14761,SFB,...,735.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
6666,2019-01-01,G4,G4,G4,801,G4,216NV,801,15295,TOL,...,945.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3333,2019-12-01,DL,DL,DL,768,DL,N695DL,768,10397,ATL,...,404.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3334,2019-12-01,DL,DL,DL,768,DL,N695DL,768,13204,MCO,...,404.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3335,2019-12-01,DL,DL,DL,769,DL,N896DN,769,10397,ATL,...,669.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3328,2019-12-01,DL,DL,DL,764,DL,N916DU,764,10397,ATL,...,669.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


In [90]:
df.fl_date = df.fl_date.apply(pd.to_datetime)

In [91]:
df.fl_date.value_counts()  

2019-02-20    6679
2019-02-18    6061
2019-02-22    5823
2019-02-12    5706
2019-02-21    5565
2019-02-19    5409
2019-02-01    5195
2019-02-10    5031
2019-02-25    5002
2019-02-14    4995
2019-02-13    4839
2019-02-08    4748
2019-02-07    4663
2019-02-11    4622
2019-02-15    4593
2019-02-17    4180
2019-02-24    4157
2019-02-27    4105
2019-02-26    3958
2019-02-06    3856
2019-02-23    3539
2019-02-04    3031
2019-02-28    2842
2019-02-16    2680
2019-02-05    2487
2019-02-02    2268
2019-02-09    2117
2019-02-03    1849
Name: fl_date, dtype: int64

In [92]:
df.groupby(df.fl_date.dt.month)['arr_delay'].mean()

fl_date
2    53.6021
Name: arr_delay, dtype: float64

In [139]:
np.std(df.groupby(df.fl_date.dt.month)['arr_delay'].mean())

3.9213125975947842

#### **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

In [54]:
df.weather_delay.isna().value_counts()

True     96189
False    23811
Name: weather_delay, dtype: int64

In [55]:
df.weather_delay.value_counts()

0.0      22655
1.0         40
2.0         32
9.0         32
15.0        31
         ...  
127.0        1
126.0        1
164.0        1
94.0         1
136.0        1
Name: weather_delay, Length: 150, dtype: int64

In [93]:
df['weather_delay'].agg(['max', 'min', 'mean', 'std', 'median'])

max       199.000000
min         0.000000
mean        2.291733
std        12.991763
median      0.000000
Name: weather_delay, dtype: float64

#### **Task 4**: How taxi times changing during the day? Does higher traffic lead to bigger taxi times?

#### **Task 5**: What is the average percentage of delays that is already created before departure? (aka are arrival delays caused by departure delays?) Are airlines able to lower the delay during the flights?

In [38]:
df[(df['dep_delay'] > 0) & (df['arr_delay'] > 0)]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
2,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,11278,DCA,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
8,2019-01-01,AA,AA_CODESHARE,AA,5615,OH,N703PS,5615,11057,CLT,...,91.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
16,2019-01-01,AA,AA_CODESHARE,AA,5647,OH,N712PS,5647,11057,CLT,...,333.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
24,2019-01-01,AA,AA_CODESHARE,AA,5658,OH,N582NN,5658,12448,JAN,...,562.0,15.0,0.0,1.0,0.0,0.0,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,2019-12-01,WN,WN,WN,4885,WN,N489WN,4885,15016,STL,...,259.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
9994,2019-12-01,WN,WN,WN,5994,WN,N483WN,5994,15016,STL,...,259.0,14.0,0.0,0.0,0.0,11.0,None,NaN,NaN,None
9995,2019-12-01,WN,WN,WN,4796,WN,N401WN,4796,15016,STL,...,440.0,0.0,0.0,0.0,0.0,59.0,None,NaN,NaN,None
9998,2019-12-01,WN,WN,WN,3137,WN,N8654B,3137,15016,STL,...,1057.0,133.0,0.0,0.0,0.0,0.0,None,NaN,NaN,None


In [39]:
df[(df['dep_delay'] > 0)]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
1,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,...,157.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,11278,DCA,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
8,2019-01-01,AA,AA_CODESHARE,AA,5615,OH,N703PS,5615,11057,CLT,...,91.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
16,2019-01-01,AA,AA_CODESHARE,AA,5647,OH,N712PS,5647,11057,CLT,...,333.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2019-12-01,WN,WN,WN,5994,WN,N483WN,5994,15016,STL,...,259.0,14.0,0.0,0.0,0.0,11.0,None,NaN,NaN,None
9995,2019-12-01,WN,WN,WN,4796,WN,N401WN,4796,15016,STL,...,440.0,0.0,0.0,0.0,0.0,59.0,None,NaN,NaN,None
9997,2019-12-01,WN,WN,WN,2609,WN,N8502Z,2609,15016,STL,...,1057.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
9998,2019-12-01,WN,WN,WN,3137,WN,N8654B,3137,15016,STL,...,1057.0,133.0,0.0,0.0,0.0,0.0,None,NaN,NaN,None


In [40]:
df[(df['arr_delay'] > 0)]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
2,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,11278,DCA,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
6,2019-01-01,AA,AA_CODESHARE,AA,5613,OH,N242JS,5613,11278,DCA,...,310.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
7,2019-01-01,AA,AA_CODESHARE,AA,5615,OH,N703PS,5615,10431,AVL,...,91.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
8,2019-01-01,AA,AA_CODESHARE,AA,5615,OH,N703PS,5615,11057,CLT,...,91.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,2019-12-01,WN,WN,WN,4885,WN,N489WN,4885,15016,STL,...,259.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
9994,2019-12-01,WN,WN,WN,5994,WN,N483WN,5994,15016,STL,...,259.0,14.0,0.0,0.0,0.0,11.0,None,NaN,NaN,None
9995,2019-12-01,WN,WN,WN,4796,WN,N401WN,4796,15016,STL,...,440.0,0.0,0.0,0.0,0.0,59.0,None,NaN,NaN,None
9998,2019-12-01,WN,WN,WN,3137,WN,N8654B,3137,15016,STL,...,1057.0,133.0,0.0,0.0,0.0,0.0,None,NaN,NaN,None


In [41]:
df[(df['dep_delay'] > 0) & (df['arr_delay'] < 0)]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
1,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,...,157.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
36,2019-01-01,AA,AA_CODESHARE,AA,5672,OH,N550NN,5672,12264,IAD,...,322.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
41,2019-01-01,AA,AA_CODESHARE,AA,5677,OH,N576NN,5677,11423,DSM,...,815.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
62,2019-01-01,AA,AA_CODESHARE,AA,5703,OH,N506AE,5703,11057,CLT,...,350.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
65,2019-01-01,AA,AA_CODESHARE,AA,5707,OH,N710PS,5707,11996,GSP,...,75.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9943,2019-12-01,WN,WN,WN,4712,WN,N446WN,4712,14908,SNA,...,342.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
9962,2019-12-01,WN,WN,WN,2883,WN,N8522P,2883,15016,STL,...,1047.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
9988,2019-12-01,WN,WN,WN,4925,WN,N557WN,4925,15016,STL,...,770.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
9989,2019-12-01,WN,WN,WN,4931,WN,N734SA,4931,15016,STL,...,770.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


#### **Task 6**: How many states cover 50% of US air traffic? 

In [31]:
df.dest_city_name.value_counts().head(17)

Chicago, IL              7003
Atlanta, GA              5584
Denver, CO               4569
Dallas/Fort Worth, TX    4241
Houston, TX              4039
Los Angeles, CA          3851
New York, NY             3759
Washington, DC           3241
Charlotte, NC            3234
Phoenix, AZ              3172
Las Vegas, NV            3096
San Francisco, CA        2539
Seattle, WA              2523
Orlando, FL              2404
Detroit, MI              2363
Baltimore, MD            2270
Minneapolis, MN          2215
Name: dest_city_name, dtype: int64

60695

#### **Task 7**: Test the hypothesis whether planes fly faster when there is the departure delay? 

#### **Task 8**: When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?

#### **Task 9**: Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

In [42]:
df.dest_city_name.value_counts().head(10)

Chicago, IL              7003
Atlanta, GA              5584
Denver, CO               4569
Dallas/Fort Worth, TX    4241
Houston, TX              4039
Los Angeles, CA          3851
New York, NY             3759
Washington, DC           3241
Charlotte, NC            3234
Phoenix, AZ              3172
Name: dest_city_name, dtype: int64

In [43]:
df.origin_city_name.value_counts().head(10).sum()

43273

#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?